In [ ]:
import os
import sys
import json
import datetime

import pareto

import pandas as pd
import numpy as np

In [ ]:
cwd = os.getcwd()
join = os.path.join
norm = os.path.normpath

In [ ]:
sys.path.append(norm(join(cwd, '..', '..', '..', 'glhe')))

In [ ]:
from standalone.plant_loop import PlantLoop
import glhe

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
plt.style.use('seaborn-bright')
plt.rcParams['figure.figsize'] = [7, 5]
plt.rcParams['font.size'] = 14

# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

In [ ]:
def run_simulation(f, dt, N_seg, N_iter, flow):
    
    d = glhe.utilities.functions.load_json('MFRTRT_STS_input.json')
    d['borehole-definitions'][0]['number-iterations'] = N_iter
    d['borehole-definitions'][0]['fraction-grout'] = f
    d['borehole-definitions'][0]['segments'] = N_seg
    
    d['ground-heat-exchanger'][0]['g-function-path'] = norm(join(cwd, 'g.csv'))
    d['simulation']['time-step'] = dt
    d['simulation']['runtime'] = 3600 * 24
    d['simulation']['output-path'] = cwd
    
    if flow == 'low':
        d['temperature-profile'][0]['path'] = norm(join(cwd, 'input_data_low.csv'))
        d['flow-profile'][0]['path'] = norm(join(cwd, 'input_data_low.csv'))
        d['simulation']['initial-temperature'] = 17.1
        d['ground-temperature-model']['temperature'] = 17.1
    else:
        d['temperature-profile'][0]['path'] = norm(join(cwd, 'input_data_high.csv'))
        d['flow-profile'][0]['path'] = norm(join(cwd, 'input_data_high.csv'))
        d['simulation']['initial-temperature'] = 16.1
        d['ground-temperature-model']['temperature'] = 16.1
    
    f_name = '{}_f-{}_dt-{}_Nseg-{}_Niter-{}'.format(flow, f, dt, N_seg, N_iter)
    
    d['simulation']['output-csv-name'] = 'out_{}.csv'.format(f_name)
    
    with open(join(cwd, 'in_{}.json'.format(f_name)), 'w') as f:
        f.write(json.dumps(d, sort_keys=True, indent=2, separators=(',', ': ')))
    
    PlantLoop('in_{}.json'.format(f_name)).simulate()

In [ ]:
dts = [30]
fs = [0.25, 0.5, 0.75]
N_segs = [1]
N_iters = [1, 2, 3]
flows = ['low', 'high']

for f in fs:
    for dt in dts:
        for N_seg in N_segs:
            for N_iter in N_iters:
                for flow in flows:
                    run_simulation(f, dt, N_seg, N_iter, flow)
                    print('f: {}, dt: {}, N_seg: {}, N_iter: {}, Flow: {}'.format(f, dt, N_seg, N_iter, flow))

In [ ]:
def process_results(f, dt, N_seg, N_iter, flow):
    
    f_name = 'out_{}_f-{}_dt-{}_Nseg-{}_Niter-{}'.format(flow, f, dt, N_seg, N_iter)

    if not os.path.exists('{}.csv'.format(f_name)):
        return 0, 0, 0
    
    if not os.path.exists('{}.txt'.format(f_name)):
        return 0, 0, 0
    
    print('f: {}, dt: {}, N_seg: {}, N_iter: {}, Flow: {}'.format(f, dt, N_seg, N_iter, flow))
    
    df = pd.read_csv('{}.csv'.format(f_name), parse_dates=True, index_col='Date/Time')

    df_data = pd.read_csv('input_data_{}.csv'.format(flow), parse_dates=True, index_col='Date/Time')
    df_data['time'] = pd.to_timedelta(df_data.index)
    df_data['time'] = datetime.datetime(year=2019, month=1, day=1, hour=0, minute=0) + (df_data['time'] - df_data['time'][0])
    df_data.set_index('time', inplace=True)
    df_data.index.rename('Date/Time', inplace=True)
    
    
    df_data.resample('{}S'.format(dt)).mean()
    df['Outlet Temp Error [C]'] = df['GroundHeatExchangerSTS:GHE 1:Outlet Temp. [C]'] - df_data['Exp. Outlet [C]']

    rmse = np.mean(df['Outlet Temp Error [C]'] ** 2) ** 0.5
    mbe = np.mean(df['Outlet Temp Error [C]'])

    mm = 0 
    ss = 0 
    ss_tot = 0

    with open('{}.txt'.format(f_name)) as f:
        for line in f:
            tokens = line.split(':')
            mm = float(tokens[-2]) * 60
            ss = float(tokens[-1])
            ss_tot = mm + ss

    return rmse, mbe, ss_tot

In [ ]:
dts = [30]
fs = [0.25, 0.5, 0.75]
N_segs = [1, 2, 3, 4]
N_iters = [1, 2, 3]
flows = ['low', 'high']

columns = ['grout-fraction', 'time step', 'number-segments', 'number-iterations', 'flow', 'rmse', 'mbe', 'time']

df_all = pd.DataFrame(columns=columns)
idx = 0

for f in fs:
    for dt in dts:
        for N_seg in N_segs:
            for N_iter in N_iters:
                for flow in flows:
                    rmse, mbe, time = process_results(f, dt, N_seg, N_iter, flow)
                    d_data = {'grout-fraction': f,
                              'time step': dt, 
                              'number-segments': N_seg,
                              'number-iterations': N_iter,
                              'flow': flow,
                              'rmse': rmse,
                              'mbe': mbe,
                              'time': time}
                    
                    df_temp = pd.DataFrame(d_data, index=[idx])
                    df_all = pd.concat([df_all, df_temp], axis=0, sort=True)
                    idx += 1

In [ ]:
df_low = df_all.loc[df_all['flow']=='low']
df_low = df_low.drop('flow', axis=1)

df_high = df_all.loc[df_all['flow']=='high']
df_high = df_high.drop('flow', axis=1)

In [ ]:
df_low.head()

In [ ]:
df_high.head()

In [ ]:
def merge(df1, df2):
    dts = [30]
    fs = [0.25, 0.5, 0.75]
    N_segs = [1, 2, 3, 4]
    N_iters = [1, 2, 3]

    columns = ['grout-fraction', 'time step', 'number-segments', 'number-iterations', 'rmse', 'mbe', 'time']
    
    df = pd.DataFrame(columns=columns)
    idx = 0
    
    for f in fs:
        for dt in dts:
            for N_seg in N_segs:
                for N_iter in N_iters:
                    rmse1 = df1['rmse'].loc[(df1['grout-fraction'] == f) &
                                            (df1['time step'] == dt) &
                                            (df1['number-segments'] == N_seg) &
                                            (df1['number-iterations'] == N_iter)].values
                    
                    mbe1 = df1['mbe'].loc[(df1['grout-fraction'] == f) &
                                          (df1['time step'] == dt) &
                                          (df1['number-segments'] == N_seg) &
                                          (df1['number-iterations'] == N_iter)].values
                    
                    time1 = df1['time'].loc[(df1['grout-fraction'] == f) &
                                            (df1['time step'] == dt) &
                                            (df1['number-segments'] == N_seg) &
                                            (df1['number-iterations'] == N_iter)].values
                    
                    rmse2 = df2['rmse'].loc[(df2['grout-fraction'] == f) &
                                            (df2['time step'] == dt) &
                                            (df2['number-segments'] == N_seg) &
                                            (df2['number-iterations'] == N_iter)].values
                    
                    mbe2 = df2['mbe'].loc[(df2['grout-fraction'] == f) &
                                          (df2['time step'] == dt) &
                                          (df2['number-segments'] == N_seg) &
                                          (df2['number-iterations'] == N_iter)].values
                    
                    time2 = df2['time'].loc[(df2['grout-fraction'] == f) &
                                            (df2['time step'] == dt) &
                                            (df2['number-segments'] == N_seg) &
                                            (df2['number-iterations'] == N_iter)].values
                    
                    if (rmse1 != 0) and (rmse2 != 0):
                        d_data = {'grout-fraction': f,
                                  'time step': dt, 
                                  'number-segments': N_seg,
                                  'number-iterations': N_iter,
                                  'rmse': np.mean([rmse1, rmse2]),
                                  'mbe': np.mean([mbe1, mbe2]),
                                  'time': np.mean([time1, time2])}

                        df_temp = pd.DataFrame(d_data, index=[idx])
                        df = pd.concat([df, df_temp], axis=0, sort=True)
                        idx += 1
    return df

In [ ]:
df_merge = merge(df_low, df_high)

In [ ]:
df_merge.head()

In [ ]:
fig = plt.figure(dpi=200)

c = df_merge['grout-fraction']
plt.scatter(df_merge['rmse'], df_merge['time'], marker='x', c=c)

plt.title('Grout Fraction')
plt.ylabel('Simulation Time [s]')
plt.xlabel('RMSE [C]')
plt.colorbar()
plt.grid()
plt.savefig('STS_Grout_Fraction.PNG', bbox_inches='tight')
plt.show()

In [ ]:
fig = plt.figure(dpi=200)

c = df_merge['time step']
plt.scatter(df_merge['rmse'], df_merge['time'], marker='x', c=c)

plt.title('Time Step')
plt.ylabel('Simulation Time [s]')
plt.xlabel('RMSE [C]')
plt.colorbar()
plt.grid()
plt.savefig('STS_Time_Step.PNG', bbox_inches='tight')
plt.show()

In [ ]:
fig = plt.figure(dpi=200)

c = df_merge['number-segments']
plt.scatter(df_merge['rmse'], df_merge['time'], marker='x', c=c)

plt.title('Segments')
plt.ylabel('Simulation Time [s]')
plt.xlabel('RMSE [C]')
plt.colorbar()
plt.grid()
plt.show()

In [ ]:
fig = plt.figure(dpi=200)

c = df_merge['number-iterations']
plt.scatter(df_merge['rmse'], df_merge['time'], marker='x', c=c)

plt.title('Iterations')
plt.ylabel('Simulation Time [s]')
plt.xlabel('RMSE [C]')
plt.colorbar()
plt.grid()
plt.savefig('STS_Iterations.PNG', bbox_inches='tight')
plt.show()

In [ ]:
def define_pareto(df_in, x_col_idx, y_col_idx):
    df = pd.DataFrame.from_records(pareto.eps_sort([list(df_in.itertuples(False))], [x_col_idx, y_col_idx]), columns=list(df_in.columns.values))
    df.sort_values(by=['rmse'], inplace=True)
    return df

In [ ]:
df_pareto = define_pareto(df_merge, 4, 5)
df_pareto.to_csv('STS_Pareto_Data.csv')

In [ ]:
df_pareto

In [ ]:
fig = plt.figure(dpi=200)

plt.scatter(df_merge['rmse'], df_merge['time'], marker='x')
plt.plot(df_pareto['rmse'], df_pareto['time'], c='r', label='Pareto')
plt.ylabel('Simulation Time [s]')
plt.xlabel('RMSE [C]')
# plt.xlim([0.24, 0.3])
# plt.ylim([30, 100])
plt.legend()
plt.grid()
plt.savefig('STS_Pareto.PNG', bbox_inches='tight')
plt.show()

In [ ]:
data_path_high = norm(join(cwd, '..', '..', 'validation', 'MFRTRT', 'MFRTRT_loads.csv'))
df_raw_data_high = pd.read_csv(data_path_high, parse_dates=True, index_col='Date/Time')
df_exp_high = df_raw_data_high[['Inst. HT [1] [W]', 'mdot [kg/s]', 'Outlet 1 [C]', 'Inlet 1 [C]']].copy(deep=True)
df_exp_high.rename(columns = {'Inst. HT [1] [W]': 'Exp. HT Rate [W]', 
                                    'Outlet 1 [C]': 'Exp. Inlet [C]', 
                                    'Inlet 1 [C]': 'Exp. Outlet [C]'}, inplace=True)
df_exp_high['time'] = pd.to_timedelta(df_exp_high.index)
df_exp_high['time'] = datetime.datetime(year=2019, month=1, day=1, hour=0, minute=0) + (df_exp_high['time'] - df_exp_high['time'][0])
df_exp_high.set_index('time', inplace=True)
df_exp_high.index.rename('Date/Time', inplace=True)
df_exp_high = df_exp_high.resample('30S').mean()

In [ ]:
data_path_low = norm(join(cwd, '..', '..', 'validation', 'MFRTRT', 'MFRTRT_Low_Flow_Loads.csv'))
df_exp_low = pd.read_csv(data_path_low, parse_dates=True, index_col='Date/Time')
df_exp_low['time'] = pd.to_timedelta(df_exp_low.index)
df_exp_low['time'] = datetime.datetime(year=2019, month=1, day=1, hour=0, minute=0) + (df_exp_low['time'] - df_exp_low['time'][0])
df_exp_low.set_index('time', inplace=True)
df_exp_low.index.rename('Date/Time', inplace=True)
df_exp_low = df_exp_low.resample('30S').mean()

In [ ]:
f = 0.5
dt = 30
N_seg = 1
N_iter = 2

df_high = pd.read_csv('out_high_f-{}_dt-{}_Nseg-{}_Niter-{}.csv'.format(f, dt, N_seg, N_iter), parse_dates=True, index_col=0)
df_low = pd.read_csv('out_low_f-{}_dt-{}_Nseg-{}_Niter-{}.csv'.format(f, dt, N_seg, N_iter), parse_dates=True, index_col=0)

In [ ]:
df_high['Outlet Temp Error [C]'] = df_high['GroundHeatExchangerSTS:GHE 1:Outlet Temp. [C]'] - df_exp_high['Exp. Outlet [C]']
df_low['Outlet Temp Error [C]'] = df_low['GroundHeatExchangerSTS:GHE 1:Outlet Temp. [C]'] - df_exp_low['Exp. Outlet [C]']

In [ ]:
start_time = '2019-01-01 00:00:00'
end_time = '2019-01-02 00:00:00'

fig = plt.figure(dpi=200)

ax = fig.add_subplot(111)

ln1 = ax.plot(df_exp_high['Exp. Inlet [C]'].loc[start_time:end_time], label='Exp. Inlet', marker='x')
ln2 = ax.plot(df_exp_high['Exp. Outlet [C]'].loc[start_time:end_time], label='Exp. Outlet', marker='x')

ln3 = ax.plot(df_high['GroundHeatExchangerSTS:GHE 1:Inlet Temp. [C]'].loc[start_time:end_time], linestyle='--', label='Sim. Inlet')
ln4 = ax.plot(df_high['GroundHeatExchangerSTS:GHE 1:Outlet Temp. [C]'].loc[start_time:end_time], linestyle='--', label='Sim. Outlet')

ax.grid()
ax.set_ylabel('Temperature [C]')

ax2 = ax.twinx()
ax2.set_ylabel('Error [C]')
ax2.set_xlabel('Date/Time')
ln5 = ax2.plot(df_high['Outlet Temp Error [C]'].loc[start_time:end_time], linestyle='-.', label='Outlet Error')

lns = ln1 + ln2 + ln3 + ln4 + ln5
labs = [l.get_label() for l in lns]
ax2.legend(lns, labs, loc=4)

plt.gcf().autofmt_xdate()

# plt.savefig('STS_High_Results_30_Min.PNG', bbox_inches='tight')
# plt.savefig('STS_High_Results_2_hr.PNG', bbox_inches='tight')
plt.savefig('STS_High_Results_24_hr.PNG', bbox_inches='tight')

plt.show()

In [ ]:
start_time = '2019-01-01 00:00:00'
end_time = '2019-01-02 00:00:00'

fig = plt.figure(dpi=200)

ax = fig.add_subplot(111)

ln1 = ax.plot(df_exp_low['Exp. Inlet [C]'].loc[start_time:end_time], label='Exp. Inlet', marker='x')
ln2 = ax.plot(df_exp_low['Exp. Outlet [C]'].loc[start_time:end_time], label='Exp. Outlet', marker='x')

ln3 = ax.plot(df_low['GroundHeatExchangerSTS:GHE 1:Inlet Temp. [C]'].loc[start_time:end_time], linestyle='--', label='Sim. Inlet')
ln4 = ax.plot(df_low['GroundHeatExchangerSTS:GHE 1:Outlet Temp. [C]'].loc[start_time:end_time], linestyle='--', label='Sim. Outlet')

ax.grid()
ax.set_ylabel('Temperature [C]')

ax2 = ax.twinx()
ax2.set_ylabel('Error [C]')
ax2.set_xlabel('Date/Time')
ln5 = ax2.plot(df_low['Outlet Temp Error [C]'].loc[start_time:end_time], linestyle='-.', label='Outlet Error')

lns = ln1 + ln2 + ln3 + ln4 + ln5
labs = [l.get_label() for l in lns]
ax2.legend(lns, labs, loc=2)

plt.gcf().autofmt_xdate()

# plt.savefig('STS_Low_Results_30_Min.PNG', bbox_inches='tight')
# plt.savefig('STS_Low_Results_2_hr.PNG', bbox_inches='tight')
plt.savefig('STS_Low_Results_24_hr.PNG', bbox_inches='tight')

plt.show()